Text classification using CamemBERT
=================================

We implemented text classification on the Allociné reviews dataset using CamemBERT.

In [ ]:
!pip install --upgrade transformers
!pip install simpletransformers
!pip install dvc
!pip install datasets

     |████████████████████████████████| 2.0MB 18.8MB/s 
     |████████████████████████████████| 890kB 55.6MB/s 
     |████████████████████████████████| 3.2MB 56.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=d33395dccd67095fb7e9c3b0f4d0b4aeef06d0ea4dcafef98177e2971b1e0c08
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 215kB 20.3MB/s 
     |████████████████████████████████| 2.0MB 50.0MB/s 
     |████████████████████████████████| 317kB 54.2MB/s 
     |████████████████████████████████| 7.0MB 43.2MB/s 
     |████████████████████████████████| 194kB 51.5MB/s 
     |████████████████████████████████| 81kB 10.0MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 1.2MB 44.2MB/s 
     |████████████████████████████████| 102kB 12.3MB/s 
     |███████████████████████████

     |████████████████████████████████| 634kB 18.5MB/s 
     |████████████████████████████████| 296kB 52.4MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 204kB 47.9MB/s 
     |████████████████████████████████| 112kB 58.0MB/s 
     |████████████████████████████████| 3.1MB 42.6MB/s 
     |████████████████████████████████| 522kB 48.0MB/s 
     |████████████████████████████████| 2.6MB 46.3MB/s 
     |████████████████████████████████| 71kB 10.5MB/s 
     |████████████████████████████████| 204kB 55.6MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 552kB 48.9MB/s 
  Created wheel for nanotime: filename=nanotime-0.5.2-cp37-none-any.whl size=2442 sha256=3a0045d9b389afce1046d773173d85a88107d1a622a27ba2a65dcd4026753f6b
  Stored in directory: /root/.cache/pip/wheels/41/99/17/7135f635215e1f61e906295afd11f4f791cfe4ab45f3bfdca2
  Created wheel fo

     |████████████████████████████████| 71kB 7.7MB/s 
  Found existing installation: tqdm 4.59.0
    Uninstalling tqdm-4.59.0:
      Successfully uninstalled tqdm-4.59.0


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from datasets import load_dataset


Load dataset

In [ ]:
dataset = load_dataset("allocine")

train_df = pd.DataFrame({'text': dataset['train']['review'][:25000], 'labels': dataset['train']['label'][:25000]})
valid_df = pd.DataFrame({'text': dataset['validation']['review'], 'labels': dataset['validation']['label']})
test_df = pd.DataFrame({'text': dataset['test']['review'], 'labels': dataset['test']['label']})
train_df

Reusing dataset allocine_dataset (/root/.cache/huggingface/datasets/allocine_dataset/allocine/1.0.0/bbee2ebb45a067891973b91ebdd40a93598d1e2dd5710b6714cdc2cd81d0ed65)


,text,labels
0,Si vous cherchez du cinéma abrutissant à tous ...,0
1,"Trash, re-trash et re-re-trash...! Une horreur...",0
2,"Et si, dans les 5 premières minutes du film, l...",0
3,Mon dieu ! Quelle métaphore filée ! Je suis ab...,0
4,"Premier film de la saga Kozure Okami, ""Le Sabr...",1
...,...,...
24995,"Quel beau film, tout y est, les acteurs sont f...",1
24996,Un documentaire émouvant sur ces millions d'am...,1
24997,Le genre de film qui entre par une oreille et ...,0
24998,73 minutes d'un scénario qui nous endore pénib...,0


Truncate dataset

In [ ]:
train_df = train_df[:25000]

,text,labels
0,Si vous cherchez du cinéma abrutissant à tous ...,0
1,"Trash, re-trash et re-re-trash...! Une horreur...",0
2,"Et si, dans les 5 premières minutes du film, l...",0
3,Mon dieu ! Quelle métaphore filée ! Je suis ab...,0
4,"Premier film de la saga Kozure Okami, ""Le Sabr...",1
...,...,...
495,"Ce film est vraiment très très bien, je me sou...",1
496,C'est très bon et complètement loufoque. Ça co...,1
497,inattendu et captivant.. bien que la réalisati...,1
498,"Je n'ai pas pu le voir en entier, mais le peu ...",1


Define model

In [ ]:
model_args = ClassificationArgs(num_train_epochs=3, overwrite_output_dir=False, manual_seed=42)
model = ClassificationModel(model_type='camembert', model_name='camembert-base', use_cuda=True, num_labels=2, args=model_args)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'cl

Train it

In [ ]:
model.train_model(train_df)

(9375, 0.1685619183776031)

Load pretrained


In [ ]:
model_args = ClassificationArgs(num_train_epochs=3, overwrite_output_dir=False, manual_seed=42)
model = ClassificationModel(model_type='camembert', model_name='outputs/checkpoint-20000', use_cuda=True, num_labels=2, args=model_args)

Evaluate its performance

In [ ]:
result, model_outputs, wrong_preds = model.eval_model(valid_df)
print(result)


{'mcc': 0.8930918460602015, 'tp': 9337, 'tn': 9593, 'fp': 611, 'fn': 459, 'auroc': 0.9875345923959715, 'auprc': 0.9860996964217978, 'eval_loss': 0.26346013724287043}


Predict on test set

In [ ]:
test_predictions, raw_outputs = model.predict(list(test.text.values))

  0%|          | 0/3263 [00:00<?, ?it/s]

  0%|          | 0/408 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Save as submission file

In [ ]:
test_original = pd.read_csv("/content/test.csv")

sample_sub = pd.DataFrame({"target": test_predictions, "id": test_original.id})
sample_sub.to_csv("submission.csv",index=False)
files.download("submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download checkpoint

In [ ]:
!zip -r checkpoint-20000x8 outputs/checkpoint-20000

  adding: outputs/checkpoint-20000/ (stored 0%)
  adding: outputs/checkpoint-20000/tokenizer_config.json (deflated 47%)
  adding: outputs/checkpoint-20000/config.json (deflated 49%)
  adding: outputs/checkpoint-20000/pytorch_model.bin (deflated 7%)
  adding: outputs/checkpoint-20000/model_args.json (deflated 62%)
  adding: outputs/checkpoint-20000/optimizer.pt (deflated 11%)
  adding: outputs/checkpoint-20000/sentencepiece.bpe.model (deflated 49%)
  adding: outputs/checkpoint-20000/training_args.bin (deflated 49%)
  adding: outputs/checkpoint-20000/scheduler.pt (deflated 49%)
  adding: outputs/checkpoint-20000/special_tokens_map.json (deflated 47%)


In [ ]:
from google.colab import files
files.download('checkpoint-20000x8.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!du -hd1

96K	./.config
1.3G	./cache_dir
15G	./outputs
52K	./runs
55M	./sample_data
18G	.
